In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor


In [4]:
%run Toolkit.ipynb

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 22.91it/s]


Number of file/s : 2
File 1 - magnets.txt
File 2 - pens.txt


In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Dict
import json
from langchain_core.agents import AgentActionMessageLog, AgentFinish
from langchain.agents.format_scratchpad import format_to_openai_function_messages

class FlashCardResponse(BaseModel):
    note: str = Field(description="A message to the user")
    flashcards: Dict[str, str] = Field(description="A dictionary of flashcards where key is the question and value is the answer")

def flashCardsAgent(human_text, verbose):
    # return ("Initialting Convo Agent")
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", '''You are an AI assistent who helps student study, student will only ask you to create Flash cards 
                        on a either a specific topic or a broad topic. You will prepared those flash cards in a json format. 
                        You will create upto 20 flash cards unless an exact number is specified by the used. If the user did 
                        not specify the exact number of cards then you will create upto 20 flash cards and attach a note in the 
                        json that informs the user that they arfe specify the exact number if they want. 
                        
                     
                        
                        
            '''),
            ("placeholder", "{chat_history}"),
            ("human", "{input}"),
            ("placeholder", "{agent_scratchpad}"),
        ]
    )


    class Response(BaseModel):
        """Final response to the question being asked"""
        note: str = Field(description="A message to the user")
        flashcards: Dict[str, str] = Field(description="A dictionary of flashcards where key is the question and value is the answer")

    def parse(output):
    # If no function was invoked, return to user
        if "function_call" not in output.additional_kwargs:
            return AgentFinish(return_values={"output": output.content}, log=output.content)
    
        # Parse out the function call
        function_call = output.additional_kwargs["function_call"]
        name = function_call["name"]
        inputs = json.loads(function_call["arguments"])
    
        # If the Response function was invoked, return to the user with the function inputs
        if name == "Response":
            return AgentFinish(return_values=inputs, log=str(function_call))
        # Otherwise, return an agent action
        else:
            return AgentActionMessageLog(
                tool=name, tool_input=inputs, log="", message_log=[output]
        )

    
    chat_model_flashcard = ChatOpenAI(model = "gpt-4-turbo")
    llm_with_tools = chat_model_flashcard.bind_functions([retriever_tool, Response])
    # agent = create_tool_calling_agent(chat_model_flashcard, tools, prompt)
    agent = (
        {
            "input": lambda x: x["input"],
            # Format agent scratchpad from intermediate steps
            "agent_scratchpad": lambda x: format_to_openai_function_messages(
                x["intermediate_steps"]
            ),
        }
        | prompt
        | llm_with_tools
        | parse
    )
    if verbose :  
        agent_executor = AgentExecutor(agent=agent, tools=[retriever_tool], verbose=True)
    else : 
        agent_executor = AgentExecutor(agent=agent, tools=[retriever_tool], verbose=False)
    result = agent_executor.invoke({"input": human_text}, return_only_outputs=True )
    return result

In [ ]:
   # The format of your output will be as defined between < >  
                        
                        # <
                        # flashCardsAgentOutput = 
                        #     note : This will contain note for the user, it can remain empty if needed,
                        #     flashCareds : 
                        #         question1 : answer1, 
                        #         question2 : answer2
                        #         ..... and so on 
                        # >